# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.3245, train_loss: 2.09008980, train_accuracy: 0.4141, test_Accuracy: 0.2966
Epoch: [ 0] [    1/  468] time: 1.4172, train_loss: 1.97520661, train_accuracy: 0.5078, test_Accuracy: 0.4465
Epoch: [ 0] [    2/  468] time: 1.5169, train_loss: 1.89607346, train_accuracy: 0.5625, test_Accuracy: 0.5594
Epoch: [ 0] [    3/  468] time: 1.6147, train_loss: 1.78933120, train_accuracy: 0.6094, test_Accuracy: 0.5905
Epoch: [ 0] [    4/  468] time: 1.7044, train_loss: 1.73967910, train_accuracy: 0.6250, test_Accuracy: 0.6179
Epoch: [ 0] [    5/  468] time: 1.8002, train_loss: 1.56646335, train_accuracy: 0.6797, test_Accuracy: 0.6344
Epoch: [ 0] [    6/  468] time: 1.8899, train_loss: 1.47619867, train_accuracy: 0.6562, test_Accuracy: 0.6589
Epoch: [ 0] [    7/  468] time: 2.0016, train_loss: 1.31420219, train_accuracy: 0.7812, test_Accuracy: 0.6929
Epoch: [ 0] [    8/  468] time: 2.0904,

Epoch: [ 0] [   73/  468] time: 8.1941, train_loss: 0.32867202, train_accuracy: 0.9141, test_Accuracy: 0.9157
Epoch: [ 0] [   74/  468] time: 8.2858, train_loss: 0.23713917, train_accuracy: 0.9141, test_Accuracy: 0.9179
Epoch: [ 0] [   75/  468] time: 8.3766, train_loss: 0.16996054, train_accuracy: 0.9453, test_Accuracy: 0.9164
Epoch: [ 0] [   76/  468] time: 8.4723, train_loss: 0.43571526, train_accuracy: 0.8984, test_Accuracy: 0.9161
Epoch: [ 0] [   77/  468] time: 8.5611, train_loss: 0.37043455, train_accuracy: 0.8828, test_Accuracy: 0.9158
Epoch: [ 0] [   78/  468] time: 8.6499, train_loss: 0.23509395, train_accuracy: 0.9453, test_Accuracy: 0.9174
Epoch: [ 0] [   79/  468] time: 8.7406, train_loss: 0.25420672, train_accuracy: 0.9141, test_Accuracy: 0.9179
Epoch: [ 0] [   80/  468] time: 8.8344, train_loss: 0.21975523, train_accuracy: 0.9297, test_Accuracy: 0.9181
Epoch: [ 0] [   81/  468] time: 8.9261, train_loss: 0.45760658, train_accuracy: 0.8672, test_Accuracy: 0.9172
Epoch: [ 0

Epoch: [ 0] [  146/  468] time: 14.9989, train_loss: 0.19441837, train_accuracy: 0.9453, test_Accuracy: 0.9379
Epoch: [ 0] [  147/  468] time: 15.0916, train_loss: 0.31131342, train_accuracy: 0.8906, test_Accuracy: 0.9387
Epoch: [ 0] [  148/  468] time: 15.1844, train_loss: 0.22314575, train_accuracy: 0.9375, test_Accuracy: 0.9364
Epoch: [ 0] [  149/  468] time: 15.2801, train_loss: 0.25475097, train_accuracy: 0.9297, test_Accuracy: 0.9348
Epoch: [ 0] [  150/  468] time: 15.3759, train_loss: 0.22558138, train_accuracy: 0.9297, test_Accuracy: 0.9335
Epoch: [ 0] [  151/  468] time: 15.4706, train_loss: 0.36778110, train_accuracy: 0.9062, test_Accuracy: 0.9347
Epoch: [ 0] [  152/  468] time: 15.5604, train_loss: 0.14065799, train_accuracy: 0.9609, test_Accuracy: 0.9367
Epoch: [ 0] [  153/  468] time: 15.6571, train_loss: 0.20463800, train_accuracy: 0.9219, test_Accuracy: 0.9331
Epoch: [ 0] [  154/  468] time: 15.7469, train_loss: 0.25749704, train_accuracy: 0.9219, test_Accuracy: 0.9323
E

Epoch: [ 0] [  220/  468] time: 22.0121, train_loss: 0.15021566, train_accuracy: 0.9453, test_Accuracy: 0.9487
Epoch: [ 0] [  221/  468] time: 22.1059, train_loss: 0.14001358, train_accuracy: 0.9531, test_Accuracy: 0.9483
Epoch: [ 0] [  222/  468] time: 22.2016, train_loss: 0.16364977, train_accuracy: 0.9688, test_Accuracy: 0.9483
Epoch: [ 0] [  223/  468] time: 22.3004, train_loss: 0.15483142, train_accuracy: 0.9609, test_Accuracy: 0.9498
Epoch: [ 0] [  224/  468] time: 22.3871, train_loss: 0.14527152, train_accuracy: 0.9688, test_Accuracy: 0.9510
Epoch: [ 0] [  225/  468] time: 22.4789, train_loss: 0.15231481, train_accuracy: 0.9453, test_Accuracy: 0.9505
Epoch: [ 0] [  226/  468] time: 22.5726, train_loss: 0.18411502, train_accuracy: 0.9531, test_Accuracy: 0.9497
Epoch: [ 0] [  227/  468] time: 22.6694, train_loss: 0.19527541, train_accuracy: 0.9609, test_Accuracy: 0.9488
Epoch: [ 0] [  228/  468] time: 22.7601, train_loss: 0.15220648, train_accuracy: 0.9531, test_Accuracy: 0.9471
E

Epoch: [ 0] [  293/  468] time: 28.8808, train_loss: 0.21359704, train_accuracy: 0.9375, test_Accuracy: 0.9539
Epoch: [ 0] [  294/  468] time: 28.9735, train_loss: 0.13582093, train_accuracy: 0.9688, test_Accuracy: 0.9493
Epoch: [ 0] [  295/  468] time: 29.0702, train_loss: 0.25662029, train_accuracy: 0.9375, test_Accuracy: 0.9471
Epoch: [ 0] [  296/  468] time: 29.1760, train_loss: 0.15638120, train_accuracy: 0.9531, test_Accuracy: 0.9468
Epoch: [ 0] [  297/  468] time: 29.2677, train_loss: 0.15583412, train_accuracy: 0.9688, test_Accuracy: 0.9491
Epoch: [ 0] [  298/  468] time: 29.3605, train_loss: 0.13568136, train_accuracy: 0.9609, test_Accuracy: 0.9516
Epoch: [ 0] [  299/  468] time: 29.4552, train_loss: 0.28368074, train_accuracy: 0.9453, test_Accuracy: 0.9541
Epoch: [ 0] [  300/  468] time: 29.5530, train_loss: 0.10468358, train_accuracy: 0.9688, test_Accuracy: 0.9566
Epoch: [ 0] [  301/  468] time: 29.6487, train_loss: 0.23490012, train_accuracy: 0.9219, test_Accuracy: 0.9585
E

Epoch: [ 0] [  367/  468] time: 35.8212, train_loss: 0.11155874, train_accuracy: 0.9531, test_Accuracy: 0.9503
Epoch: [ 0] [  368/  468] time: 35.9229, train_loss: 0.15319988, train_accuracy: 0.9688, test_Accuracy: 0.9445
Epoch: [ 0] [  369/  468] time: 36.0147, train_loss: 0.13970019, train_accuracy: 0.9453, test_Accuracy: 0.9455
Epoch: [ 0] [  370/  468] time: 36.1094, train_loss: 0.20020905, train_accuracy: 0.9375, test_Accuracy: 0.9522
Epoch: [ 0] [  371/  468] time: 36.2002, train_loss: 0.19374210, train_accuracy: 0.9375, test_Accuracy: 0.9555
Epoch: [ 0] [  372/  468] time: 36.2929, train_loss: 0.18781735, train_accuracy: 0.9141, test_Accuracy: 0.9554
Epoch: [ 0] [  373/  468] time: 36.3867, train_loss: 0.13122033, train_accuracy: 0.9531, test_Accuracy: 0.9546
Epoch: [ 0] [  374/  468] time: 36.4794, train_loss: 0.13437997, train_accuracy: 0.9688, test_Accuracy: 0.9525
Epoch: [ 0] [  375/  468] time: 36.5782, train_loss: 0.08651699, train_accuracy: 0.9766, test_Accuracy: 0.9533
E

Epoch: [ 0] [  441/  468] time: 42.6848, train_loss: 0.09820770, train_accuracy: 0.9766, test_Accuracy: 0.9629
Epoch: [ 0] [  442/  468] time: 42.7746, train_loss: 0.04165828, train_accuracy: 1.0000, test_Accuracy: 0.9628
Epoch: [ 0] [  443/  468] time: 42.8673, train_loss: 0.11949582, train_accuracy: 0.9688, test_Accuracy: 0.9616
Epoch: [ 0] [  444/  468] time: 42.9581, train_loss: 0.03453850, train_accuracy: 0.9922, test_Accuracy: 0.9613
Epoch: [ 0] [  445/  468] time: 43.0469, train_loss: 0.21163216, train_accuracy: 0.9453, test_Accuracy: 0.9614
Epoch: [ 0] [  446/  468] time: 43.1376, train_loss: 0.13312984, train_accuracy: 0.9688, test_Accuracy: 0.9621
Epoch: [ 0] [  447/  468] time: 43.2314, train_loss: 0.15779206, train_accuracy: 0.9297, test_Accuracy: 0.9646
Epoch: [ 0] [  448/  468] time: 43.3181, train_loss: 0.06806149, train_accuracy: 0.9844, test_Accuracy: 0.9648
Epoch: [ 0] [  449/  468] time: 43.4209, train_loss: 0.08039498, train_accuracy: 0.9766, test_Accuracy: 0.9647
E

# Test accuracy : 96.55 %